In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

The optical description of the sea state by the capillary wave statistics is a good approximation to nature particularly for seas freshly activated by winds. However it is to be expected that the optical properties of a fully developed sea, over which for long times and extensive fetches the wind has been active, may be affected by the presence of the resultance gravity waves. In particular, at low solar altitudes the effects of wave shielding may become important, since the incoming rays will tend to strike the frontsides of the large gravity waves while the backsides remain in shadow

A simple hybrid wave model is possible that will combine capillary and gravity waves. We first construct the gravity wave swells over a rectangular grid using the air-water surface representations - this defines a set of gravity wave facets. The capillary hexagonal grid is then fitted to each gravity wave facet. Shielding by the gravity waves can be thought of  a change in the incoming ray direction relative to the capillary waves riding on the gravity wave facet. Thus when a ray strikes a gravity wave facet which is tilted toward the sun, the capillary waves on that facet see the sun as being "higher" in the sky relative to the plane of their gravity wave facet. 

![rectangular grid](images/rectangular_grid.png)
The rectangular, spatial-domain coordinate system used to construct a random air-water surface of triangular gravity-wave facets. The facet triads are shown by dashed lines

e.g. spatial region is $X \times Y = 25 \times 25 m$, which is resolved by $N_x \times N_y = 2048 \times 1024$ grid points. The smallest resolved wavelength in the x-direction is then $2 \times 25/2048 = 0.024 m$, and twice that in the crosswind diection. Thus this particular surface resolves all wavelengths from 25 m gravity waves almost down to capillary waves, which have wavelengths of order 1 cm.


The $\omega$ th realization of a spatially stationary, zero-mean random surface over this grid is defined by a surface elevation function $\eta$ whose value at node ($i,j$) is:

$$\eta (i;j;\omega) = \sum_{(u,v) \in W} \left[b_{uv}(\omega) \cos 2 \pi \left(\frac{ui}{p} + \frac{vj}{q} \right) + c_{uv} \sin 2 \pi \left(\frac{ui}{p} + \frac{vj}{q} \right) \right]$$

Here $W$ is the set of $(u,v)$ values such that $u = 0$ and $1 \leq v \leq m$, or $1 \leq u \leq l$ and $-m \leq v \leq m$; $p=2l + 1$ and $q = 2m + 1$

The quantities $b_{uv}$ and $c_{uv}$ are independent, normally distributed random variables of zero mean and variances:

$\Epsilon \{ b_{uv}^2\} = \Epsilon \{ c_{uv}^2\} = E(k_u,k_v)\Delta u \Delta v$,

where $\Delta u = \frac{2 \pi}{p \Delta x}, \quad \Delta v = \frac{2 \pi}{q \Delta y}$

$k_u = u\Delta u, \quad k_v = v \Delta v$

The quantity $E(k_u,k_v)$ is the directional energy spectrum of the waves in rectangular coordinate form. $E(k_u,k_v)$ has the symmetry $E(-k_u,-k_v) = E(k_u,k_v)$ so that $W$ covers all wave trains moving generally downwind

$$E(k_u, k_v) = F(\sigma_{uv}, \phi_{uv}) \frac{g^2}{2 \sigma_{uv}^3} \quad (m^4)$$

where

$\sigma_{uv} = (g k_{uv}^{1/2}) \quad (s^{-1})$

$\phi_{uv} = \tan^{-1} (\frac{k_v}{k_u})$

$k_{uv} = (k_u^2 + k_v^2)^{1/2} \quad (m^{-1})$

$F(\sigma_{uv}, \phi_{uv}) = C \sigma_{uv}^{-6} \exp \left[ -2 \left( \frac{g}{\sigma_{uv} U} \right)^2 \right] f(\sigma_{uv}, \phi_{uv}) \quad (m^2 s)$

$f(\sigma_{uv}, \phi_{uv}) = 1 + (0.50 + 0.82 \xi) \cos (2 \phi_{uv}) + 0.32 \xi \cos (2 \phi_{uv})$

$\xi = \exp \left[ -\frac{1}{2} \left(\frac{\sigma_{uv} U}{g} \right)^4 \right]$

$C = 0.763 \quad m^2 s^{-5}$ and $g = 9.80 \quad m s^{-2}$

Here $\sigma_{uv}$ is temporal wave frequency,

$k_{uv}$ is spatial wave number of the deep water gravity waves, and $g$ is gravity's acceleration

This formidable set of equations is used as follows. 

Select the wind speed $U$ (m/s). This fixes the frequency of the highest spectral energy density of waves represented by the spectrum: $\sigma_{max} = (\frac{2}{3})^{1/2} \frac{g}{U}$

By $\sigma^2 = gk$ and $\lambda = 2 \pi/k$, the value of $\sigma_{max}$ also determines the associated wavelength of the waves of maximum energy density $\lambda_{max} = \frac{3 \pi U^2}{g}$

This wavelength can be used to set the physical scale of the hexagonal grid. e.g. $X = Y = 4 \lambda_{max}$ would give a hexagonal domain capable of resolving four wavelengths of the maximum-energy waves. Each of the maximum-energy waves is resolved according the choice of $l$ and $m$: e.g. $l = m = 24$, gives $\Delta x = \Delta y = \lambda_{max}/24$. Note that the choice of $\delta = 1$ no longer holds. Once the grid resolution has been fixed, $E{k_u,k_v}$ is determined and evaluated for each pair of integers $(u,v)$ in $W$. This calculation is performed only once.

Finally, to construct the $\omega$ th realization of a random surface, for each $(u,v) \in W$, we randomly draw (for the $\omega$ th time) independent samples $b_{uv}(\omega)$ and $c_{uv}(\omega)$ from a normal distribution of zero mean and variance $E(k_u,k_v) \Delta u \Delta v$. Then we obtain $\eta$ from the above equations.

Note that the vertices of the wave triads, used to define the triangular wave facets, coincide with every other $(i,j)$ point of the rectangular grid used to define $E (k_u,k_v)$. Thus, $\eta$ need to be evaluated only at those ($i,j$) nodes coinciding with a triad vertex.

![spectral domain coord](images/spectral_domain_coord.png)

The spectral-domain $(u,v)$ coordinate system corresponding to the spatial domain. $W$ is the set of $(u,v)$ values shown by circles

In [47]:
# maximum wavelength of waves of maximm energy density
U = 6
g = 9.8

lambda_max = 3*np.pi*U**2/g

# Hexagonal domain capable of resolving 4 wavelengths of the maximum-energy waves
X = Y = 4*lambda_max
l = m = 24 #increase this parameter if you want higher resolution
delta_x = delta_y = lambda_max/l

p = 2*l+1 #number of grids incl 0
q = 2*m + 1 #number of grids incl 0

delta_u = 2*np.pi/(p*delta_x)
delta_v = 2*np.pi/(q*delta_y)

print(delta_u)
print(delta_v)
# k_u = lambda u: u*delta_u
# k_v = lambda v: v*delta_v

C = 0.763

0.0888888888888889
0.0888888888888889


In [58]:
v = np.linspace(0, m, m+1,dtype=int)
u = np.repeat(0,m+1)

v1 = np.linspace(m,-m,q,dtype=int)
u1 = np.linspace(1,l,l,dtype=int)
U1,V1 = np.meshgrid(u1,v1)

v = np.concatenate((v,V1.flatten()))
u = np.concatenate((u,U1.flatten()))

k_u = u*delta_u
k_v = v*delta_v
k_uv = np.sqrt(k_u**2 + k_v**2)

sigma_uv = (g*k_uv)**0.5

phi_uv = np.arctan(k_v/k_u)

phi_uv

c:\Users\xinmatrix\anaconda3\envs\image_segmentation\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
c:\Users\xinmatrix\anaconda3\envs\image_segmentation\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


array([        nan,  1.57079633,  1.57079633, ..., -0.82884906,
       -0.80667155, -0.78539816])

In [59]:
xi = np.exp(-0.5*(sigma_uv*U/g)**4)
f = 1+(0.50 + 0.82*xi)*np.cos(2*phi_uv) + 0.32*xi*np.cos(4*phi_uv)
F = C*(sigma_uv**-6)*np.exp(-2*(g/(sigma_uv*U))**2)*f 
E = F*g**2/(2*sigma_uv**3)
E

c:\Users\xinmatrix\anaconda3\envs\image_segmentation\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\xinmatrix\anaconda3\envs\image_segmentation\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\xinmatrix\anaconda3\envs\image_segmentation\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until


array([           nan, 3.87101524e-03, 1.35306538e-02, ...,
       8.42765400e-06, 7.88302401e-06, 7.36164791e-06])

# Using FFT

Suppose that we have measured the sea surface elevation $\eta (x_r,y_s) = \eta (r,s) $ at a given time and over a rectangular grid of spatial points $(x_r,y_s)$, $r = 1,..., N_x$, $s = 1,...,N_y$. This grid covers an area of some physical size $X \times Y$, e.g. $0 \leq x \leq X$, $0 \leq y \leq Y$, with a grid spacing of $\Delta x = X/N_x$ and $\Delta y = Y/N_y$. The longest wavelengths resolvable by this grid are $X$ and $Y$ in the respective directions, and the smallest waves are of wavelength $2 \Delta x = 2X/N_x$ and $2 \Delta y = 2Y/N_y$

Linear wave theory says that this surface can be represented as a sum of sinusoids:

$$\eta (x_r,y_s) = \frac{a_0}{2} + \sum_{u=1}^{N_x/2} \sum_{v=1}^{N_y/2} [a_{u,v} \cos (k_u x_r + k_v y_s) + b_{u,v} \sin (k_u x_r + k_v y_s)]$$

($a_0$ represents mean sea level and can be set to 0)

where the $a_{u,v}$ and $b_{u,v}$ are the amplitudes of the waves with spatial frequencies $k_u = 2 \pi u/X$ and $k_v = 2 \pi v/Y$; $k_u$ and $k_v$ have units of radian per meter. This decomposition can be placed in the form of a discrete Fourier transform pair:

$$ \eta (r,s) = \sum_{u=0}^{N_x-1} \sum_{v=0}^{N_y-1} \eta (u,v) \exp [i 2 \pi (ru/N_x + sv/N_y)] \quad (1)$$

$$ \eta (u,v) = \frac{1}{N_x N_y} \sum_{r=0}^{N_x-1} \sum_{v=0}^{N_y-1} \eta (r,s) \exp [-i 2 \pi (ru/N_x + sv/N_y)] \quad (2)$$

(2) is called the DFT, and (1) is the inverse DFT. If $N_x,N_y$ are both powers of 2, then FFT can be used. Note that although $\eta (r,s)$ is real, $\hat{\eta} (u,v)$ is in general a complex number. However, $\hat{\eta} (u,v)$ is Hermitian, which means that $\hat{\eta}^* (u,v) = \hat{\eta} (-u,-v)$, where * represents the complex conjugate

The energy of a wave is proportional to the square of its amplitude. Thus the square of $\hat{\eta}(u,v)$ is proportional to the energy contained in waves with frequencies of $k_u, k_v$ in the respective $x,y$ directions. The quantity $E(k_u,k_v) = \parallel \hat{\eta}(u,v) \parallel ^2 = \hat{\eta} (u,v) \hat{\eta}^* (u,v)$ is therefore known as the wave energy spectrum

To use a given energy spectrum $E(k_u, k_v)$ to generate a random surface realisation $\eta (r,s)$, we first choose the $X,Y$ and $N_x, N_y$ values which will determine the physical size of the spatial region and the fineness of the wave resolution. This gives an x-y grid of points at which $\eta (r,s)$ is to be determined. Then for each $(u,v)$ combination, draw independent random numbers $\rho_1(u,v)$ and $\rho_2 (u,v)$ from a $N(0,1)$ distribution, and form the quantity

$$\hat{\eta}_o (u,v) = \frac{1}{\sqrt{2}}[\rho_1 (u,v) + i \rho_2 (u,v)] \sqrt{\frac{E(k_u,k_v)}{2}}$$

However, since $\hat{\eta}_o (u,v)$ is not Hermitian. However, the quantity

$$\hat{\eta} (u,v) = \frac{1}{2} [\hat{\eta}_o (u,v) + \hat{\eta}_o^* (u,v)]$$

is Hermitian, and thus gives a sea surface realisation that is real and consistent with the chosen wave energy spectrum

In [ ]:
N_x = 64
N_y = 64
